In [ ]:
# this code is written on google colab, with all required files stored in the user's google drive folder
# please replace the file path with your own

### Importing modules ###
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install ipympl
import matplotlib.image as mpimg # base image
!pip install basemap
from mpl_toolkits.basemap import Basemap # matplotlib map overlay
!pip install shapely
from shapely.geometry import Point, Polygon, LineString # polygon data extraction
from numpy.linalg import eig
!pip install obspy
from obspy.imaging.beachball import beachball

!pip install geopy
from geopy.distance import geodesic
from pyproj import Geod # area for strain rates
from shapely import wkt
from math import radians, sin, cos, sqrt, atan2, degrees
!pip install pyrocko
from pyrocko import moment_tensor as pmt

from google.colab import output
output.enable_custom_widget_manager()



drive.mount('/content/drive')

In [ ]:
### read file with mw and m0 ###
# read full CMT file from google drive
file_path = "/content/drive/My Drive/Colab Notebooks/cascadiafull3_mw_m0.xy" # remember to mount drive
# 2823 from file, but 2040 when summing up types of earthquakes (ie strikeslip, normal, subduction
headers = ["lon", "lat", "depth", "mrr", "mtt", "mpp", "mrt", "mrp", "mtp", "iexp", "X", "Y", "name", "mw_mttk", "m0_mttk"]
cascadia_full = pd.read_csv(file_path, delim_whitespace=True, names=headers, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
cascadia_full.drop(cascadia_full.columns[[10, 11, 12]], axis=1, inplace=True) # no name
cascadia_full["iexp"] = pd.to_numeric(cascadia_full["iexp"], downcast='float')
cascadia_full["m0_mttk"] = pd.to_numeric(cascadia_full["m0_mttk"], downcast='float')
# converts 64bit data to 32bit float (stores exponent and decimal as opposed to entire number), courtesy to a friend pointing out why its not working


### read files with CMT seperated earthquake types seperately (normal, strikeslip & subduction) ###
# too lazy to append m0 into my files
file_path = "/content/drive/My Drive/Colab Notebooks/cascadianormal_mw.xy"
headers = ["lon", "lat", "depth", "mrr", "mtt", "mpp", "mrt", "mrp", "mtp", "iexp", "X", "Y", "name", "mw_mttk"]
cascadia_normal = pd.read_csv(file_path, delim_whitespace=True, names=headers, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
cascadia_normal.drop(cascadia_normal.columns[[10, 11, 12]], axis=1, inplace=True) # no name
cascadia_normal["iexp"] = pd.to_numeric(cascadia_normal["iexp"], downcast='float')

print('cascadia_normal len', len(cascadia_normal))

file_path = "/content/drive/My Drive/Colab Notebooks/cascadiastrikeslip_mw.xy"
headers = ["lon", "lat", "depth", "mrr", "mtt", "mpp", "mrt", "mrp", "mtp", "iexp", "X", "Y", "name", "mw_mttk"]
cascadia_strikeslip = pd.read_csv(file_path, delim_whitespace=True, names=headers, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
cascadia_strikeslip.drop(cascadia_strikeslip.columns[[10, 11, 12]], axis=1, inplace=True) # no name
cascadia_strikeslip["iexp"] = pd.to_numeric(cascadia_strikeslip["iexp"], downcast='float')

print('cascadia_strikeslip len', len(cascadia_strikeslip))

file_path = "/content/drive/My Drive/Colab Notebooks/cascadiasubduction_mw.xy"
headers = ["lon", "lat", "depth", "mrr", "mtt", "mpp", "mrt", "mrp", "mtp", "iexp", "X", "Y", "name", "mw_mttk"]
cascadia_subduction = pd.read_csv(file_path, delim_whitespace=True, names=headers, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
cascadia_subduction.drop(cascadia_subduction.columns[[10, 11, 12]], axis=1, inplace=True) # no name
cascadia_subduction["iexp"] = pd.to_numeric(cascadia_subduction["iexp"], downcast='float')

print('cascadia_subduction len', len(cascadia_subduction))


# here for each row in each df, we check whether this earthquake exists in cascadia_full (it should!)
# then we append a value (normal=1; strikeslip=2; subduction=3) in the new "type" column in cascadia_full

# new column in cascadia_full called "type"
cascadia_full['type'] = 0
count = 0
for row in range(len(cascadia_normal)):
  match = (cascadia_normal.iloc[row, 0:10].values == cascadia_full.iloc[:, 0:10].values).all(axis=1) # outputs boolean
  if match.any():
    count += 1
    cascadia_full.loc[match, 'type'] = 1

print("count normal", count)

count = 0
for row in range(len(cascadia_strikeslip)):
  match = (cascadia_strikeslip.iloc[row, 0:10].values == cascadia_full.iloc[:, 0:10].values).all(axis=1) # outputs boolean
  if match.any():
    count += 1
    cascadia_full.loc[match, 'type'] = 2

print("count strikeslip", count)

count = 0
for row in range(len(cascadia_subduction)):
  match = (cascadia_subduction.iloc[row, 0:10].values == cascadia_full.iloc[:, 0:10].values).all(axis=1) # outputs boolean
  if match.any():
    count += 1
    cascadia_full.loc[match, 'type'] = 3

print(f"count subduction {count}\n")


print(cascadia_full)

# Mw = moment magnitude; M0 = scaler moment = mu D A
# "type": normal=1; strikeslip=2; subduction/thrust=3
# 2823 from file, but 2040 when summing up types of eqs. this code overwrites eqs with percisely 45deg plunge with subduction
     

In [ ]:
### functions and stuff i stole from mttk ###
# stolen and edited from mttk add_m0

## function to get eigen values and vectors
def eigen(mrr, mtt, mpp, mrt, mrp, mtp):
  Mij = np.matrix([[ mtt, -mtp,  mrt],
                         [ -mtp, mpp, -mrp],
                         [ mrt, -mrp,  mrr]])

  #Calculate eigenvalues and associated eigenvectors
  eig_vals, eig_vecs = np.linalg.eig(Mij)

  idx = eig_vals.argsort()[::-1]
  eig_vals = eig_vals[idx]
  eig_vecs = eig_vecs[:,idx]

  return eig_vals, eig_vecs



## function to get m0 for a moment tensor
def get_m0(mrr, mtt, mpp, mrt, mrp, mtp, expon):
    Mij = np.matrix([[ mtt, -mtp,  mrt],
                         [ -mtp, mpp, -mrp],
                         [ mrt, -mrp,  mrr]])

    #Calculate eigenvalues and associated eigenvectors
    eig_vals, eig_vecs = np.linalg.eig(Mij)

    #Form diagonalised moment tensor (of elements with units dyne-cm)
    eig_vals = eig_vals*(10**expon)
    diag_Mij = np.diag(eig_vals)

    #Calculate scalar seismic moment:
    moment_iso   = np.trace(diag_Mij)/3.
    moment_dev   = max([j - moment_iso for j in eig_vals])
    moment_total = moment_iso + moment_dev

    #Calculate moment magnitude from dyne-cm (e.g. Kanamori & Hanks 1979):
    Mw = np.log10(moment_total)*(2./3) - 10.7

    return moment_total

'''
## homemade m0 code THAT ALIGNS WITH HAVARD CMT for box m0

def get_m0(mrr, mtt, mpp, mrt, mrp, mtp):
  moment_tensor = np.array([[mtt, -mtp, mrt], [-mtp, mpp, -mrp], [mrt, -mrp, mrr]])
  evals, evecs = eig(moment_tensor)
  # https://www.geeksforgeeks.org/numpy-linalg-eig-method-in-python/
  m0 = np.sqrt(0.5*(evals[0]**2 + evals[1]**2 + evals[2]**2)) # Bowers & Hudson (1999) Eqn 5
  return m0
'''

In [ ]:
### Loading GMT basemap and overlaying matplotlib map for interactive point picking ###
## seting up figure and enabling interactive plot
%matplotlib inline
%matplotlib widget

# fig = plt.figure(figsize=(15.25, 12.3))
fig = plt.figure(figsize=(10.5, 8.2)) #a4 portrait figure size
ax1 = plt.subplot(111)


## import GMT-generated basemap png
gmtmap_image = mpimg.imread("/content/drive/My Drive/Colab Notebooks/focal_mechanisms_CMT_mercator_cropped.png")


## creating matplotlib basemap for overlay
# https://matplotlib.org/basemap/index.html
m = Basemap(projection='merc', resolution='l',\
                          llcrnrlat=38.5,urcrnrlat=66.3,\
                          llcrnrlon=-180, urcrnrlon=-118.5)

# interactive_map.drawcoastlines() # drawing coastlines on matplotlib disabled


## plotting parallels and meridians, annotating axis
parallels = np.arange(30, 80, 10)
meridians = np.arange(-190, -110, 10)
m.drawparallels(parallels, labels=[1,0,0,0], fontsize=12, linewidth=0.5) # label parallels on right and top
m.drawmeridians(meridians, labels=[0,0,0,1], fontsize=12, linewidth=0.5) # meridians on bottom and left


## plotting image in bottom layer - https://stackoverflow.com/questions/11487797/python-matplotlib-basemap-overlay-small-image-on-map-plot
m.imshow(gmtmap_image, origin='upper')

plt.show()


### Interactive point picking ###
# mainly follows this - https://sgcorcoran.github.io/Python-for-MSE/lessons_3114/06_selecting_pixel_coordinates.html
# useful reference - https://matplotlib.org/stable/gallery/event_handling/pick_event_demo.html


## conversion codes - between x y and lon lat
# references this - https://matplotlib.org/basemap/users/mapcoords.html
# lon,lat can be scalars, lists or numpy arrays

def pos_to_lonlat(x, y):
  lon, lat = m(x, y, inverse=True)
  return lon, lat

def lonlat_to_pos(lon, lat):
  x, y = m(lon, lat)
  return x, y


## motified from https://sgcorcoran.github.io/Python-for-MSE/lessons_3114/06_selecting_pixel_coordinates.html
pos = [] # in format [[None, None], [x, y], [x, y], ...]
lonlat = []


def onclick(event):
    pos.append([event.xdata, event.ydata])

    lon, lat = pos_to_lonlat(pos[-1][0], pos[-1][1]) # pos[-1] represent last click (list with x, y)
    lonlat.append([lon, lat]) # converts x y to lon lat and appends

    ax1.set_title(f'Click {len(pos)}: {lon}, {lat}')


cid=fig.canvas.mpl_connect('button_press_event', onclick)
     

In [ ]:
### manual data input from excel spreedsheet ###
# REMEMBER TO PRESS RESET FIRST
lon_excel = '-155.83456221 -156.36359447 -156.43917051 -156.43917051 -156.66589862 -156.89262673 -157.27050691 -157.0437788  -155.91013825 -155.68341014 -155.6078341  -155.30552995 -154.32304147 -153.340553   -152.50921659 -152.20691244 -151.82903226 -151.45115207 -150.84654378 -150.77096774 -149.78847926 -148.27695853 -147.14331797 -146.23640553 -145.93410138 -147.97465438 -148.73041475 -148.88156682 -149.41059908 -149.93963134 -150.31751152 -150.84654378 -151.90460829 -152.05576037 -152.35806452 -152.88709677 -153.340553   -153.71843318 -154.32304147 -154.24746544 -154.47419355 -154.47419355 -155.30552995 -155.38110599'
lat_excel = '55.91381866 55.74402509 55.44509968 55.23018503 55.10067606 54.92734241 54.40283436 54.31475695 54.18228652 54.40283436 54.70962033 54.8403944 55.18706211 55.35927371 55.65885013 55.78654298 55.87143974 55.95615131 56.37693896 56.46054402 56.75171707 57.40886108 58.09438961 58.84527923 59.0790819  59.73295455 59.0790819  58.92338966 58.68852849 58.33324697 58.37290045 58.13431081 57.73308242 57.44954678 57.28653242 57.32735393 57.24566558 57.32735393 57.0406504  56.87581932 56.79313016 56.46054402 56.25118657 56.0828719'
lon_excel = [float(lon) for lon in lon_excel.split()]
lat_excel = [float(lat) for lat in lat_excel.split()]

# check whether length is the same
length_check = len(lon_excel) if len(lon_excel) == len(lat_excel) else "Lengths are not the same"
print("Number of polygon bounding points:", length_check)

lonlat = list(zip(lon_excel, lat_excel))
polygon_df = pd.DataFrame(lonlat)

print(polygon_df)


## conversion

m = Basemap(projection='merc', resolution='l',\
                          llcrnrlat=38.5,urcrnrlat=66.3,\
                          llcrnrlon=-180, urcrnrlon=-118.5)

def pos_to_lonlat(x, y):
  lon, lat = m(x, y, inverse=True)
  return lon, lat

def lonlat_to_pos(lon, lat):
  x, y = m(lon, lat)
  return x, y


pos = [lonlat_to_pos(lon, lat) for lon, lat in zip(lon_excel, lat_excel)]
pts = np.array(pos[:])
pts = np.append(pts,[pts[0]], axis=0) # to close our polygon when plotting. axis=0 specify along rows

ax1.plot(pts[:,0],pts[:,1], 'b-')



'''
file_path = "/content/drive/My Drive/Colab Notebooks/Project_Cascadia_Kostrov.xlsx"
headers = ["name", "info", "to_do", "polygon_pts", "lon", "lat", "area", "volume", "eq_no", "max_depth", "mmr", "mtt", "mpp", "mrt", "mrp", "mtp", "summed_m0", "individual_m0", "Cs",\
           "eigenP", "eigenI", "eigenT", "strike", "dip", "lon_l", "lat_l", "total_l", "w_length", "v_ms", "v_mmyr", "strainP", 'strainI', 'strainT']
excel_full = pd.read_excel(file_path, names=headers, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,\
                                                                                        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

print(excel_full)
'''
     

In [ ]:
### polygon drawing and data saving ###
pts = np.array(pos[:])
pts = np.append(pts,[pts[0]], axis=0) # to close our polygon when plotting. axis=0 specify along rows

ax1.plot(pts[:,0],pts[:,1], 'b-')

polygon_df = pd.DataFrame(lonlat)

print("Number of polygon bounding points:", len(lonlat))
print(polygon_df)

print("\n for excel")
print("longitude:")
print(polygon_df[0].to_numpy())
print("latitude:")
print(polygon_df[1].to_numpy())


print("\n for gmt; lon lat:")
for row in range(len(polygon_df)):
  print(str(polygon_df.iloc[row, 0]), str(polygon_df.iloc[row, 1]))


In [ ]:
### reset selection ###
pos = []
lonlat = []
ax1.set_title(f'Clicks reset')

for line in ax1.lines:
        line.remove()
plt.show()

In [ ]:
### input moment tensor from CMT based on box chosen ###

## check if a point is inside the polygon
polygon = Polygon(lonlat)
def is_inside_polygon(lon, lat):
    point = Point(lon, lat)
    return polygon.contains(point)

## empty df for earthquakes in polygon
headers = ["lon", "lat", "depth", "mrr", "mtt", "mpp", "mrt", "mrp", "mtp", "iexp", "X", "Y", "name", "mw_mttk", "m0_mttk", "type"]
earthquakes_in_polygon_df = pd.DataFrame(columns=headers)
earthquakes_in_polygon_df.drop(earthquakes_in_polygon_df.columns[[10, 11, 12]], axis=1, inplace=True) # no name


## extract data within the polygon
for index, row in cascadia_full.iterrows():
    # iterrows() returns both the index and pd series
    lon, lat = row['lon'], row['lat']
    if is_inside_polygon(lon, lat):
        earthquakes_in_polygon_df = earthquakes_in_polygon_df.append(row, ignore_index=True)

earthquakes_in_polygon_df_backup = earthquakes_in_polygon_df.copy() # backup


### defining depth range and earthquake type if applicable ###

min_depth = 0.
max_depth = 50
# earthquake_type = 3   # normal=1; strikeslip=2; subduction/thrust=3


earthquakes_in_polygon_df = earthquakes_in_polygon_df_backup.copy() # reset earthquake_in_polygon_df

earthquakes_in_polygon_df_filtered = earthquakes_in_polygon_df[\
(earthquakes_in_polygon_df['depth'] >= min_depth) & (earthquakes_in_polygon_df['depth'] <= max_depth)] # depth filter

# earthquakes_in_polygon_df_filtered = earthquakes_in_polygon_df_filtered[(earthquakes_in_polygon_df['type'] == 3)] # type filter
# earthquakes_in_polygon_df_filtered = earthquakes_in_polygon_df_filtered[(earthquakes_in_polygon_df['type'] == 2) | (earthquakes_in_polygon_df['type'] == 1)] # 2 types filter

earthquakes_in_polygon_df = earthquakes_in_polygon_df_filtered # confusing but so that I dont have to rename all of my variables

print("\n No. of eqs enclosed within polygon:", len(earthquakes_in_polygon_df_backup))

print(f'\nMax earthquake depth WITHOUT depth filter: {earthquakes_in_polygon_df_backup["depth"].max()} km')
polygon_max_depth = earthquakes_in_polygon_df["depth"].max()
print(f'Max earthquake depth w depth filter: {polygon_max_depth} km')

print("\n No. of eqs enclosed within polygon AFTER depth and type filter:", len(earthquakes_in_polygon_df))
print("\n Filtered df", earthquakes_in_polygon_df)

In [ ]:
### Kostrov tensor summation ###
# empty 1x7 array [mrr, mtt, mpp, mrt, mrp, mtp] to store summation
tensor_sum = np.zeros_like(earthquakes_in_polygon_df.iloc[0, 3:9]) # np.zeros_like allows the use of ref array

## polygon summation
for row in range(len(earthquakes_in_polygon_df)):
  tensor_row = earthquakes_in_polygon_df.iloc[row, 3:9] * (10**earthquakes_in_polygon_df.iloc[row, 9]) #multiply by iexp ie normalized
  tensor_sum += tensor_row

# reduce exponent of tensor_sum by 27, add iexp at the end
tensor_sum /= 10**27



## m0 from INDIVIDUAL moment tensor summation
'''
sum_of_individual_m0 = 0.
for row in range(len(earthquakes_in_polygon_df)):
  sum_of_individual_m0 += earthquakes_in_polygon_df.iloc[row, 11]
'''


## prints
print("\n Summed moment tensor Mij (* 10^27):")
print(tensor_sum)


m0_from_sum = get_m0(*tensor_sum, 27)
print("\n m0 from SUMMED moment tensor Mij:")
print(m0_from_sum)
# func is 6 different inputs not an array --> * unpacks the array into seperate enteries
# 1.5923

sum_of_individual_m0 = np.sum(earthquakes_in_polygon_df['m0_mttk'])
print("\n m0 from INDIVIDUAL moment tensor summation:")
print(sum_of_individual_m0)


Cs = m0_from_sum / sum_of_individual_m0
print("\n seismic consistency:")
print(Cs)


eigen_vals = eigen(*tensor_sum)[1]
print("\nEigen values in descending order:")
print(eigen_vals)

eigen_vec = eigen(*tensor_sum)[0]
print("Eigen vectors:")
print(eigen_vec)


## plotting beach ball
# https://docs.obspy.org/tutorial/code_snippets/beachball_plot.html
tensor_sum_list = tensor_sum.tolist()
beachball(tensor_sum_list, size=200, linewidth=2, facecolor='c')
fig.canvas.mpl_disconnect(cid) # disconnect interactive picking

In [ ]:
### plotting CUMULATIVE log(frequency)-magnitude plot ###
# tested with small sample - ok

fig, ax2 = plt.subplots(figsize =(3.5, 2.5))


magnitude_bin = np.arange(4, 10, 0.5) # min, max (stops at 9.3), interval
# barely any datapoints --> adjust histogram bin width
bin_centers = 0.5*(magnitude_bin[:-1] + magnitude_bin[1:]) # midpoints, starts at 0 ends right before the end etc

frequency_bin = np.zeros_like(bin_centers)


for row in range(len(earthquakes_in_polygon_df)):
    mw = earthquakes_in_polygon_df.iloc[row]['mw_mttk']
    for bin in range(len(magnitude_bin) - 1):
      if mw >= magnitude_bin[bin]:
        frequency_bin[bin] += 1


## plotting
ax2.plot(bin_centers, frequency_bin, 'c.-')
ax2.set_title(f'{len(earthquakes_in_polygon_df)} samples, bin width {magnitude_bin[1] - magnitude_bin[0]}')
ax2.set_yscale('log') # log10N = a − bM

# linear regression to find line of best fit; log10N = a - bM
coefficients = np.polyfit(bin_centers[2:5], np.log10(frequency_bin[2:5] + 1e-7), 1) # + 1 to avoid dividing by 10
a, b = coefficients[0], coefficients[1]
print('best fit coefficients',a,b)

x = np.linspace(4, 9.2, 100)
best_fit_line = 10 ** (b + a * x)
ax2.plot(x, best_fit_line, 'k-', label=f'log10N = {a} - {b}M')
ax2.plot([3,9],[1,1])

ax2.set_ylim(1, None) # adjusting limit
ax2.set_xlim(4, 9.2)
plt.show()

In [ ]:
### interactive fault L picking - copy paste from earlier without comments ###
## DOES NOT clip automatically within box - use eye
%matplotlib inline
%matplotlib widget

# fig = plt.figure(figsize=(15.75, 12.3))
fig = plt.figure(figsize=(10.5, 8.2))
ax4 = plt.subplot(111)

gmtmap_image = mpimg.imread("/content/drive/My Drive/Colab Notebooks/focal_mechanisms_CMT_mercator_cropped.png")

m = Basemap(projection='merc', resolution='l',\
                          llcrnrlat=38.5,urcrnrlat=66.3,\
                          llcrnrlon=-180, urcrnrlon=-118.5)


parallels = np.arange(30, 80, 10)
meridians = np.arange(-190, -110, 10)
m.drawparallels(parallels, labels=[1,0,0,0], fontsize=12, linewidth=0.5) # label parallels on right and top
m.drawmeridians(meridians, labels=[0,0,0,1], fontsize=12, linewidth=0.5) # meridians on bottom and left

ax4.plot(pts[:,0],pts[:,1], 'b-') # showing original box

m.imshow(gmtmap_image, origin='upper')
ax4.set_title(f'Pick fault length')
plt.show()

## Interactive point picking
## motified from https://sgcorcoran.github.io/Python-for-MSE/lessons_3114/06_selecting_pixel_coordinates.html
pos_fault = [] # in format [[None, None], [x, y], [x, y], ...]
lonlat_fault = []

def onclick(event):
    pos_fault.append([event.xdata, event.ydata])

    lon, lat = pos_to_lonlat(pos_fault[-1][0], pos_fault[-1][1]) # pos[-1] represent last click (list with x, y)
    lonlat_fault.append([lon, lat]) # converts x y to lon lat and appends

    ax4.set_title(f'Click {len(pos_fault)}: {lon}, {lat}')


cid=fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
## fault L saving and printing
fault_pts = np.array(pos_fault[:])

ax4.plot(fault_pts[:,0],fault_pts[:,1], 'r-') # print fault_L

fault_L_df = pd.DataFrame(lonlat_fault)
print(fault_L_df)

print("\n for excel")
print("longitude fault L:")
print(fault_L_df[0].to_numpy())
print("latitude fault L:")
print(fault_L_df[1].to_numpy())

In [ ]:
### reset selection fault_L ###
pos_fault = []
lonlat_fault = []
ax4.set_title(f'Clicks reset')

for line in ax4.lines:
        line.remove()

ax4.plot(pts[:,0],pts[:,1], 'b-') # showing original box

plt.show()

In [ ]:
### calculating velocity rate ###
# expect values orders of mm to cm a year

# calculating dustance between two points with haversine - https://en.wikipedia.org/wiki/Haversine_formula
def haversine_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2]) # convert from deg to rad

    # radius of the Earth in meters
    R = 6371000.0

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1


    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    distance = 2 * R * np.arcsin(sqrt(a))

    return distance


## fault length
# distance between all consecutive points for fault L

total_distance_fault_L = 0

for i in range(len(lonlat_fault) - 1):
    lat1, lon1 = lonlat_fault[i]
    lat2, lon2 = lonlat_fault[i + 1]
    distance = haversine_distance(lat1, lon1, lat2, lon2)
    total_distance_fault_L += distance
    # print(f"Distance between point {i + 1} and point {i + 2}: {distance:.2f} m")

dip = 27.03 # dip in deg FROM MTTK
polygon_max_depth = earthquakes_in_polygon_df_backup["depth"].max()
# polygon_max_depth = 20

print(f'\nMax earthquake depth WITHOUT depth filter: {earthquakes_in_polygon_df_backup["depth"].max()} km')
# polygon_max_depth
print(f'Max depth used: {polygon_max_depth} km')
print("Dip used:", dip)
print(f"\nTotal fault length: {total_distance_fault_L:.2f} m")

## fault W
length_W = (polygon_max_depth * 1000) / np.sin(dip * (np.pi / 180)) # km to m; sin in radians
print(f"\nlength_W: {length_W:.2f} m")



# other variables
shear_modulus = 3.3 * (10 ** 10) # Stacey (1992)
catalogue_duration_s = 17460 * 24 * 60 * 60
# from 1st Jan 1976 to 20th Oct 2023; 17460 days




v_factor = shear_modulus * total_distance_fault_L * length_W * catalogue_duration_s
v_rate = (1/ v_factor) * sum_of_individual_m0 * (1e-7) # either one m0 works; CONVERT FROM dyn cm to Nm

print(f'\nvelocity rate: {v_rate} m s^-1')

## in mm per year (to compare with catalogues)
v_rate_mmyr = v_rate * 31536000 * 1000 # 1 yr = 31536000 sec; convert from m to mm


print(f'velocity rate: {v_rate_mmyr} mm year^-1')

In [ ]:
### calculating average strain rate de / dt ###
# expect values approx 10 ** -8 or even 10 ** -11
print(f'\nMax earthquake depth WITHOUT depth filter: {earthquakes_in_polygon_df_backup["depth"].max()} km')

polygon_max_depth = earthquakes_in_polygon_df["depth"].max()
polygon_max_depth = 30

print(f'Max depth used: {polygon_max_depth} km')


## variables
shear_modulus = 3.3 * (10 ** 10) # Stacey (1992)
catalogue_duration_s = 17460 * 24 * 60 * 60
# from 1st Jan 1976 to 20th Oct 2023; 17460 days

# calculating polygon area (geodesic)
# https://stackoverflow.com/questions/23697374/calculate-polygon-area-in-planar-units-e-g-square-meters-in-shapely
# first order approx checked from manually approximated google earth pro
geod = Geod(ellps='WGS84') # around meter accuracy

polygon_wkt = wkt.loads(f"{Polygon(lonlat)}")
polygon_area = abs(geod.geometry_area_perimeter(polygon_wkt)[0])
print('\nGeodesic area: {:.3f} m^2'.format(polygon_area))

# calculating volume

polygon_volume = polygon_max_depth * 1000 * polygon_area # V = L * b * d; km to m
print('Polygon volume: {:.3f} m^3'.format(polygon_volume))


## calculating strain rate
strain_factor = 2 * shear_modulus * polygon_volume * catalogue_duration_s
strain_rate_eigen_vec = [0, 0, 0]

print('\nStrain eigen value')
for i in range(len(eigen_vec)):
  print(eigen_vec[i])
  strain_rate_eigen_vec[i] = (1 / strain_factor) *  eigen_vec[i] * (10 ** 27) * (1e-7)

# strain_rate_eigen_vec = (1 / strain_factor) *  eigen_vec * (10 ** 27) * (1e-7) # Mij doesnt have exponent information; CONVERT FROM dyn cm to Nm

print(f'\nstrain rate: {strain_rate_eigen_vec} s^-1')

strain_rate_eigen_vec_yr = [0, 0, 0]
for i in range(len(strain_rate_eigen_vec)):
  strain_rate_eigen_vec_yr[i] = strain_rate_eigen_vec[i] * 31536000

print(f'strain rate: {strain_rate_eigen_vec_yr} s^-1 year^-1')
# takes
     

In [ ]:
### strain P T eigen value surface projection ###
# normal box 2a
strain_eigen_P_yr = strain_rate_eigen_vec_yr[0]
strain_eigen_T_yr = strain_rate_eigen_vec_yr[2]
strain_eigen_P_dip = 7.27
strain_eigen_T_dip = 19.12


eig_P_projection = strain_eigen_P_yr * np.cos(abs(strain_eigen_P_dip) * (np.pi / 180))
eig_T_projection = strain_eigen_T_yr * np.cos(abs(strain_eigen_T_dip) * (np.pi / 180))

print("Strain rate eigen vector P:", strain_eigen_P_yr)
print("Strain rate eigen vector T:", strain_eigen_T_yr)
print("Eigen P dip:", strain_eigen_P_dip)
print("Eigen T dip:", strain_eigen_T_dip)


print("\nEigen P surface projection:", eig_P_projection)
print("Eigen T surface projection:", eig_T_projection)

In [ ]:
# generating a grid of lon lat points for unavco geodetic velocity vectors

lonlat_geodetic_df = pd.DataFrame(columns=["lon", "lat", "depth"])
lon1 = 118
lon2 = 180
lat1 = 38
lat2 = 66

lon_range = lon2 - lon1
print("lon range", lon_range)
lat_range = lat2 - lat1
print("lat range", lat_range)

for lon in range(lon1, lon2+1, 2): # 2deg interval
  for lat in range(lat1, lat2+1, 2):
    lonlat_geodetic_df = lonlat_geodetic_df.append({'lon': -lon, 'lat': lat, 'depth': 0.}, ignore_index=True) # append returns a new df


print("lon lat:")
for row in range(len(lonlat_geodetic_df)):
  print(str(lonlat_geodetic_df.iloc[row, 0]), str(lonlat_geodetic_df.iloc[row, 1]), str(lonlat_geodetic_df.iloc[row, 2]), end=',')
